reference: [Custom Named Entity Recognition with BERT.ipynb](https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT_only_first_wordpiece.ipynb)

In [ ]:
!pip install transformers==3.1.0
!pip install seqeval[gpu]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 884 kB 15.3 MB/s 
     |████████████████████████████████| 1.3 MB 66.1 MB/s 
     |████████████████████████████████| 3.0 MB 60.7 MB/s 
     |████████████████████████████████| 880 kB 71.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=5ed86141d054f03561d976a96a8e278a6d88445bdc351eab2e6963088353beeb
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 763 kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=69a4aa280486a8aa2df2bf2e1ad00796f7ba601c75c10f572f3bc92c9c678576
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5

In [ ]:
import pandas as pd
import csv
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
WORKDIR = '/content/drive/MyDrive/Data Mining/'
labels_to_ids = {'X': 0, 'O': 1}
ids_to_labels = {0 : 'X', 1 : 'O'}
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
df_train = pd.read_csv(f'{WORKDIR}/Batch_answers - train_data (no-blank).csv' )

# drop unnecessary columns
df_train = df_train.drop(['Unnamed: 6', 'total no.: 7987'], axis=1)

# remove punctuation
df_train['q'] = df_train['q'].str.replace("[^\w\s]", "", regex=True)
df_train['r'] = df_train['r'].str.replace("[^\w\s]", "", regex=True)
df_train["q'"] = df_train["q'"].str.replace("[^\w\s]", "", regex=True)
df_train["r'"] = df_train["r'"].str.replace("[^\w\s]", "", regex=True)
df_train = df_train.drop_duplicates()

In [ ]:
q_tag = []
r_tag = []
for index, row in df_train.iterrows():
  i = 0
  tag = []
  row_q = row['q'].split()
  row_q_ = row['q\''].split()

  for word in row_q:
    if i < len(row_q_) and word == row_q_[i]:
      tag.append("O")
      i += 1
    else:
      tag.append("X")
  q_tag.append(",".join(tag))

  i = 0
  tag = []
  row_r = row['r'].split()
  row_r_ = row['r\''].split()
  for word in row_r:
    if i < len(row_r_) and word == row_r_[i]:
      tag.append("O")
      i += 1
    else:
      tag.append("X")
  r_tag.append(",".join(tag))

df_train['q_tag'] = q_tag
df_train['r_tag'] = r_tag
df_train = df_train.dropna()
df_train.to_csv(f"{WORKDIR}/train_tag.csv")

## Load Data

In [ ]:
df_train = pd.read_csv(f'{WORKDIR}/train_tag.csv')
df_train.head()

,Unnamed: 0,id,q,r,s,q',r',q_tag,r_tag
0,0,8,It can go both ways We all doubt It is what ...,True,AGREE,It can go both ways We all doubt It is what ...,True,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O",O
1,1,8,It can go both ways We all doubt It is what ...,True,AGREE,can go both ways We all doubt It is what you...,True,"X,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O",O
2,2,8,It can go both ways We all doubt It is what ...,True,AGREE,It can go both ways We all doubt It is what ...,True,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O",O
3,3,9,once again you seem to support the killing of...,based on the idea that people are dispensible ...,AGREE,seem to support the killing of certain people,based on the idea that people are dispensible ...,"X,X,X,O,O,O,O,O,O,O,O,X,X,X","O,O,O,O,O,O,O,O,O,O,O,O,O,O,X,X,X,X,X,X,X,X,X,..."
4,4,9,once again you seem to support the killing of...,based on the idea that people are dispensible ...,AGREE,you seem to support the killing of certain peo...,based on the idea that people are dispensible,"X,X,O,O,O,O,O,O,O,O,O,O,O,O","O,O,O,O,O,O,O,O,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,..."


In [ ]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence1 = self.data.q[index].split() 
        sentence2 = self.data.r[index].split()  
        word_labels1 = self.data.q_tag[index].split(",") 
        word_labels2 = self.data.r_tag[index].split(",") 
        word_labels = word_labels1 + word_labels2

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(text=sentence1, text_pair=sentence2,
                             is_pretokenized=True, 
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)

        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            if idx >= len(encoded_labels) or i >= len(labels):
              print(len(encoded_labels), idx, len(labels), i)
            else:
              encoded_labels[idx] = labels[i]
              i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

In [ ]:
train_size = 0.8
df_train = df_train.dropna()
train_dataset = df_train.sample(frac=train_size,random_state=200)
test_dataset = df_train.drop(train_dataset.index).reset_index(drop=True)

train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df_train.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (35990, 9)
TRAIN Dataset: (28792, 9)
TEST Dataset: (7198, 9)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
model.to(device)

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(0.7412, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 512, 2])

In [ ]:
train_dataset.to_csv('/content/drive/MyDrive/Data Mining/train_final.csv', index=False)
test_dataset.to_csv('/content/drive/MyDrive/Data Mining/test_final.csv', index=False)

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 0.7285466194152832
Training loss per 100 training steps: 0.5905203656985027
Training loss per 100 training steps: 0.5818683785289082
Training loss per 100 training steps: 0.579086341731176
Training loss per 100 training steps: 0.5756631836183648
Training loss per 100 training steps: 0.571352375838571
Training loss per 100 training steps: 0.5652602423794456
Training loss per 100 training steps: 0.5641931500099865
Training loss per 100 training steps: 0.5626302980788787
Training loss per 100 training steps: 0.5566451328741459
Training loss per 100 training steps: 0.5523711258208716
Training loss per 100 training steps: 0.5493868058318988
Training loss per 100 training steps: 0.5473467205014455
Training loss per 100 training steps: 0.5428456073651214
Training loss per 100 training steps: 0.5431445636743312
Training loss per 100 training steps: 0.5416887711378672
Training loss per 100 training steps: 0.5398740086478788
Training loss p

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.6707772612571716
Validation loss per 100 evaluation steps: 0.546352075763268
Validation loss per 100 evaluation steps: 0.5325814496670196
Validation loss per 100 evaluation steps: 0.5194861913143203
Validation loss per 100 evaluation steps: 0.5125434695931147
Validation loss per 100 evaluation steps: 0.5116870125789128
Validation loss per 100 evaluation steps: 0.5145789960557331
Validation loss per 100 evaluation steps: 0.515901985441397
Validation loss per 100 evaluation steps: 0.5126573547683703
Validation loss per 100 evaluation steps: 0.5116917266614827
Validation loss per 100 evaluation steps: 0.513226112665294
Validation loss per 100 evaluation steps: 0.5163162544396765
Validation loss per 100 evaluation steps: 0.5184853334572294
Validation loss per 100 evaluation steps: 0.5156822018067623
Validation loss per 100 evaluation steps: 0.5166311554137423
Validation loss per 100 evaluation steps: 0.5165365343835256
Validation loss per 100 eva

In [ ]:
import os

directory = f"{WORKDIR}/model_final"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)

## Predict

In [ ]:
model = BertForTokenClassification.from_pretrained(f"{WORKDIR}/model_final")

In [ ]:
import re
def pred(q, r):
  q = re.sub("[^\w\s]", "", q)
  r = re.sub("[^\w\s]", "", r)

  q = q.split()
  r = r.split()
  
  inputs = tokenizer(text=q, text_pair=r,
                      is_pretokenized=True, 
                      return_offsets_mapping=True, 
                      padding='max_length', 
                      truncation=True, 
                      max_length=MAX_LEN,
                      return_tensors="pt")
  
  # move to gpu
  ids = inputs["input_ids"].to(device)
  mask = inputs["attention_mask"].to(device)
  ids = inputs["input_ids"]
  mask = inputs["attention_mask"]
  # forward pass
  outputs = model(ids, attention_mask=mask)
  logits = outputs[0]

  active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
  flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

  tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
  token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
  wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

  prediction = []
  for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
    #only predictions on first word pieces are important
    if mapping[0] == 0 and mapping[1] != 0:
      prediction.append(token_pred[1])
    else:
      continue

  q_ = []
  r_ = []
  for i in range(len(q)):
    if i < len(prediction) and prediction[i] == 'O':
      q_.append(q[i])
  
  for i in range(len(r)):
    if len(q)+i < len(prediction) and prediction[len(q)+i] == 'O':
      r_.append(r[i])
  
  return q_, r_


In [ ]:
test = pd.read_csv(f"{WORKDIR}/Batch_answers - test_data(no_label).csv", encoding='unicode_escape')

In [ ]:
test['q'] = test['q'].str.replace("[^\w\s]", "", regex=True)
test['r'] = test['r'].str.replace("[^\w\s]", "", regex=True)
test.head()

,id,q,r,s
0,6199,Time,If so why do we still have apes and why are ...,DISAGREE
1,5807,There s a lot of discussion there on that issu...,Of course The makers of Expelled were within ...,DISAGREE
2,8487,It s not helping The guns these people have ...,Oh I would wager about like Mexico about 80 ...,DISAGREE
3,1760,Shooting 3 seriously injured in Auburn shooti...,Pickup strikes group of four youths Houston ...,AGREE
4,6228,This is the argument concerning choice that ...,I believe there is a point at which we societ...,DISAGREE


In [ ]:
ans = []
i = 0
for index, row in test.iterrows():
    q_, r_ = pred(row['q'], row['r'])
    q_ans = " ".join(q_)
    r_ans = " ".join(r_)
    ans.append([row['id'], q_ans, r_ans])

fields = ['id', 'q', 'r'] 
  
with open(f'{WORKDIR}/submission_final.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(ans)

In [ ]:
out = pd.read_csv(f"{WORKDIR}/submission_final.csv", encoding='unicode_escape')
out = out.sort_values(by=['id'])
out.to_csv(f"{WORKDIR}/submission_final.csv", index=False)